In [2]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"  # Opens the plot in a web browser
import numpy as np


In [33]:
epsilon_values = [ round(e,3) for e in np.arange(0.005, 1.0, 0.005).tolist()]
epsilon_values

[0.005,
 0.01,
 0.015,
 0.02,
 0.025,
 0.03,
 0.035,
 0.04,
 0.045,
 0.05,
 0.055,
 0.06,
 0.065,
 0.07,
 0.075,
 0.08,
 0.085,
 0.09,
 0.095,
 0.1,
 0.105,
 0.11,
 0.115,
 0.12,
 0.125,
 0.13,
 0.135,
 0.14,
 0.145,
 0.15,
 0.155,
 0.16,
 0.165,
 0.17,
 0.175,
 0.18,
 0.185,
 0.19,
 0.195,
 0.2,
 0.205,
 0.21,
 0.215,
 0.22,
 0.225,
 0.23,
 0.235,
 0.24,
 0.245,
 0.25,
 0.255,
 0.26,
 0.265,
 0.27,
 0.275,
 0.28,
 0.285,
 0.29,
 0.295,
 0.3,
 0.305,
 0.31,
 0.315,
 0.32,
 0.325,
 0.33,
 0.335,
 0.34,
 0.345,
 0.35,
 0.355,
 0.36,
 0.365,
 0.37,
 0.375,
 0.38,
 0.385,
 0.39,
 0.395,
 0.4,
 0.405,
 0.41,
 0.415,
 0.42,
 0.425,
 0.43,
 0.435,
 0.44,
 0.445,
 0.45,
 0.455,
 0.46,
 0.465,
 0.47,
 0.475,
 0.48,
 0.485,
 0.49,
 0.495,
 0.5,
 0.505,
 0.51,
 0.515,
 0.52,
 0.525,
 0.53,
 0.535,
 0.54,
 0.545,
 0.55,
 0.555,
 0.56,
 0.565,
 0.57,
 0.575,
 0.58,
 0.585,
 0.59,
 0.595,
 0.6,
 0.605,
 0.61,
 0.615,
 0.62,
 0.625,
 0.63,
 0.635,
 0.64,
 0.645,
 0.65,
 0.655,
 0.66,
 0.665,
 0.67,
 

In [31]:
epsilon_values = np.linspace(0.001, 0.995, num=199).tolist()
epsilon_values

[0.001,
 0.00602020202020202,
 0.011040404040404041,
 0.01606060606060606,
 0.021080808080808082,
 0.026101010101010104,
 0.031121212121212122,
 0.03614141414141414,
 0.04116161616161616,
 0.046181818181818185,
 0.05120202020202021,
 0.05622222222222222,
 0.061242424242424244,
 0.06626262626262626,
 0.07128282828282828,
 0.0763030303030303,
 0.08132323232323233,
 0.08634343434343435,
 0.09136363636363637,
 0.09638383838383839,
 0.10140404040404041,
 0.10642424242424242,
 0.11144444444444444,
 0.11646464646464647,
 0.12148484848484849,
 0.1265050505050505,
 0.13152525252525252,
 0.13654545454545455,
 0.14156565656565656,
 0.1465858585858586,
 0.1516060606060606,
 0.15662626262626264,
 0.16164646464646465,
 0.16666666666666666,
 0.1716868686868687,
 0.1767070707070707,
 0.18172727272727274,
 0.18674747474747475,
 0.19176767676767678,
 0.1967878787878788,
 0.20180808080808083,
 0.20682828282828283,
 0.21184848484848484,
 0.21686868686868688,
 0.22188888888888889,
 0.22690909090909092,
 0.

In [17]:
file_path = '/home/yyj/Desktop/yyj/thesis/code/PETSQL/data/process/PPL_DEV.JSON-9_SHOT_Euclidean_mask_1034_1/results_4.json'

In [18]:

# 讀取 JSON 檔案
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

In [19]:
import pandas as pd
import plotly.express as px

# 初始化 DataFrame
records = []

for entry in data:
    index = entry["index"]
    weights = entry["current_weights"]
    for model, weight in weights.items():
        records.append({"index": index, "model": model, "weight_value": weight})

df = pd.DataFrame(records)

# 確保數據格式正確並轉換為長格式
df_melted = df.pivot(index="index", columns="model", values="weight_value").reset_index().melt(
    id_vars=["index"], var_name="model", value_name="weight_value"
)

# 按 index 分組，對 weight_value 進行排序，新增 rank 欄位
df_melted["rank"] = df_melted.groupby("index")["weight_value"].rank(ascending=False, method="first")

# 按 index 和 weight_value 排序，確保 hover 顯示時權重由大到小排列
df_melted = df_melted.sort_values(by=["index", "weight_value"], ascending=[True, False])

# 使用 Plotly 繪製互動式折線圖
fig = px.line(
    df_melted,
    x="index",
    y="weight_value",
    color="model",
    markers=True,
    title="Dynamic Trend of Model Weights",
    hover_data={"model": True, "weight_value": ":.6f", "rank": True}  # 顯示 rank
)

fig.update_layout(
    xaxis_title="Index",
    yaxis_title="Weight",
    hovermode="x unified",  # 讓 hover 時所有模型資訊一起顯示
    template="plotly_dark",
    legend_title="Model"
)

# 顯示互動式圖表
fig.show()


In [9]:
# 初始化 DataFrame
records = []

for entry in data:
    index = entry["index"]
    weights = entry["current_weights"]
    for model, weight in weights.items():
        records.append({"index": index, "model": model, "weight": weight})

df = pd.DataFrame(records)

# 確保數據格式正確
df_pivot = df.pivot(index="index", columns="model", values="weight").reset_index()

# 這裡修改 value_name，避免與原始欄位名稱衝突
df_melted = df.pivot(index="index", columns="model", values="weight").reset_index().melt(
    id_vars=["index"], var_name="model", value_name="weight_value"
)

# 使用 Plotly 繪製可互動的折線圖
fig = px.line(
    df_melted,
    x="index",
    y="weight_value",
    color="model",
    markers=True,
    title="Dynamic Trend of Model Weights"
)

fig.update_layout(
    xaxis_title="Index",
    yaxis_title="Weight",
    hovermode="x unified",
    template="plotly_dark"
)

# 顯示互動式圖表
fig.show()

## final answer

In [3]:
final_path = '/home/yyj/Desktop/yyj/thesis/code/PETSQL/data/process/PPL_DEV.JSON-9_SHOT_Euclidean_mask_1034_1/final_result_no_cc.json'
# 讀取 JSON 檔案
with open(final_path, "r", encoding="utf-8") as f:
    final_data = json.load(f)

In [4]:
# Extract `final_sql` values
final_sql_statements = [entry["final_sql"] for entry in final_data]
output_txt_file = '/home/yyj/Desktop/yyj/thesis/code/PETSQL/data/process/PPL_DEV.JSON-9_SHOT_Euclidean_mask_1034_1/final_sql_no_cc.txt'

# Write to a text file
with open(output_txt_file, "w", encoding="utf-8") as f:
    for sql in final_sql_statements:
        f.write(sql + "\n")

print(f"Successfully written {len(final_sql_statements)} SQL statements to {output_txt_file}")

Successfully written 1034 SQL statements to /home/yyj/Desktop/yyj/thesis/code/PETSQL/data/process/PPL_DEV.JSON-9_SHOT_Euclidean_mask_1034_1/final_sql_no_cc.txt


In [21]:
output_txt_file_1 = '/home/yyj/Desktop/yyj/thesis/code/PETSQL/data/process/PPL_DEV.JSON-9_SHOT_Euclidean_mask_1034_3/epsilon/final_sql_0.01.txt'
# Write to a text file
with open(output_txt_file_1) as f:
    output_1 = [l.strip().split('\t') for l in f.readlines() if len(l.strip()) > 0]
output_txt_file_2 = '/home/yyj/Desktop/yyj/thesis/code/PETSQL/data/process/PPL_DEV.JSON-9_SHOT_Euclidean_mask_1034_3/epsilon/final_sql_0.005.txt'
# Write to a text file
with open(output_txt_file_2) as f:
    output_2 = [l.strip().split('\t') for l in f.readlines() if len(l.strip()) > 0]
same = False
if output_1 == output_2:
    same = True

In [22]:
same

False